In [1]:
import numpy as np
import numpy.linalg as npl

import scipy as sp
import scipy.linalg as spl
import scipy.sparse as sps
import scipy.sparse.linalg as spsl
import matplotlib.pyplot as plt

%matplotlib widget

pi = np.pi

# Introduction EDP : Méthodes différences finies

## Équation de Poisson

Soit l'équation de Poisson sur $\Omega = ]0,1[$,
$$-u''(x) = f(x), \forall x \in ]0,1[$$
avec les conditions aux bords de Dirichlet homogène, c'est à dire $u(0) = u(1) = 0$

In [2]:
def f1(x):
    return pi**2 * np.sin(pi * x) 

def u_exacte_f1(N, u0=0, u1=0):
    x = np.linspace(0, 1, N)
    return np.sin(pi * x) + (u1-u0)*x + u0

In [3]:
dx = 0.001
x = np.arange(0, 1, step=dx)
plt.figure()
plt.plot(x, f1(x))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def mat_A(N):
    diags = [-1, 2, -1]
    return sps.diags(diags, offsets=[-1, 0, 1], shape=[N,N], format="csr")

def u_approx(f, N, u0=0, u1=0):
    x = np.linspace(0, 1, N+2)[1:-1]
    dx = 1/(N+1)
    A = mat_A(N)/(dx**2)
    c = np.zeros(N)
    c[0], c[-1] = u0, u1
    
    U = spsl.spsolve(A, f(x) + c/(dx**2))
    return U

In [5]:
def J(n):
    return 2**n - 1

def extend_vect(vect, u0=0, u1=0):
    ext_vect = np.zeros(vect.size+2)
    ext_vect[1:-1] = vect
    ext_vect[0], ext_vect[-1] = u0, u1
    return ext_vect    

In [6]:
def approx_sol(n, f, u0=0, u1=0):
    N = J(n)
    U = u_approx(f, N, u0, u1)
    U = extend_vect(U, u0, u1)
    return U

def plot_sol(U):
    N = U.size
    x = np.linspace(0, 1, N)
    plt.plot(x, U, label=f"{int(np.log2(N+1))}")

def plot_range(f, u_exacte, low_bound=3, up_bound=8,
                 precision=10000, u0=0, u1=0):
    plt.figure()
    plot_sol(u_exacte(precision, u0, u1))
    
    for n in range(low_bound, up_bound):
        U = approx_sol(n, f, u0, u1)
        plot_sol(U)
        
    plt.legend(loc="best")
    plt.show()

In [7]:
def erreur(f, u_exacte, n, ordre=2, u0=0, u1=0):
    N = J(n)
    U = u_approx(f, N, u0, u1)
    U = extend_vect(U, u0, u1)
    u = u_exacte(N+2, u0, u1)
    e = npl.norm(u - U, ordre)
    return e

def erreur_range(f, u_exacte, low_bound=3, up_bound=8, u0=0, u1=0):
    e2 = []
    einf = []
    r = range(low_bound, up_bound)
    for n in r:
        e2.append(erreur(f, u_exacte, n, 2, u0, u1))
        einf.append(erreur(f, u_exacte, n, np.inf, u0, u1))
        
    plt.figure()
    plt.semilogy(list(r), e2, label="Norme 2")
    plt.semilogy(list(r), einf, label="Norme infinie")
    plt.legend(loc="best")
    plt.show()

In [8]:
plot_range(f1, u_exacte_f1, low_bound=3, up_bound=7, u0=0, u1=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
erreur_range(f1, u_exacte_f1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def f2(x):
    return 25 * pi**2 * np.sin(5 * pi * x)

def u_exacte_f2(N, u0=0, u1=0):
    x = np.linspace(0, 1, N)
    return np.sin(5 * pi * x)

In [11]:
plot_range(f2, u_exacte_f2, low_bound=3, up_bound=6, u0=0, u1=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
erreur_range(f2, u_exacte_f2, low_bound=3, up_bound=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
def f3(x):
    return np.where(x<0.5, 1, 0)

def u_exacte_f3(N, u0=0, u1=0):
    x = np.linspace(0, 1, N)
    a = 3/8
    return np.where(x<0.5, -(x**2)/2 + a*x, (0.25 - a)*x - 0.25 + a)

In [14]:
N = 10000
x = np.linspace(0, 1, N)
plt.figure()
plt.plot(x, f3(x))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_range(f3, u_exacte_f3, low_bound=3, up_bound=6, u0=0, u1=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
erreur_range(f3, u_exacte_f3, low_bound=3, up_bound=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
plt.close('all')

## Exo 2 : Equation de la chaleur

### Schéma explicite

In [17]:
def u0(x):
    return np.sin(pi*x)

In [18]:
dx = 0.1
dt = 0.002
T = 0.7

In [19]:
def plot_sharex(dts, sch, dx, T):
    n = len(dts)
    plt.figure()
    for i, dt in enumerate(dts, 1):
        U = sch(u0, dt, dx, T)
        if i==1:
            ax1 = plt.subplot(n, 1, i)
            ax = ax1
        else:
            ax = plt.subplot(n, 1, i, sharex=ax1)
        for k in range(len(U)):
            p = plt.plot(U[k])
        plt.legend([f"dt = {dt}"])
        plt.setp(ax.get_xticklabels(), visible=i==n)
        
    plt.show()

In [46]:
def sch_explicite(u0, dt, dx=0.1, T=0.2):
    N = int(1/dx) - 1
    x = np.linspace(0, 1, N+2)
    U = [u0(x)]
    n = 0
    A = dt*mat_A(N).toarray()/(dx**2)

    while n*dt <= T:
        u1 = np.zeros_like(U[-1])
        u1[1:-1] = U[-1][1:-1] - A.dot(U[-1][1:-1])
        U.append(u1)
        n += 1
        
    return U

In [21]:
plot_sharex([0.002, 0.01], sch_explicite, dx=0.1, T=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
def sch_implicite(u0, dt, dx=0.1, T=0.2):
    N = int(1/dx) - 1
    x = np.linspace(0, 1, N+2)
    U = [u0(x)]
    n = 0
    A = dt*mat_A(N).toarray()/(dx**2)

    while n*dt <= T:
        u1 = np.zeros_like(U[-1])
        u1[1:-1] = npl.solve(np.eye(N) + A, U[-1][1:-1])
        U.append(u1)
        n += 1
        
    return U

In [23]:
plot_sharex([0.002, 0.01], sch_implicite, dx=0.1, T=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
def theta_sch(u0, dt, dx=0.1, T=0.2, theta=1/2):
    N = int(1/dx) - 1
    x = np.linspace(0, 1, N+2)
    U = [u0(x)]
    n = 0
    A = dt*mat_A(N).toarray()/(dx**2)

    while n*dt <= T:
        u1 = np.zeros_like(U[-1])
        p_explicite = (np.eye(N) - theta*A).dot(U[-1][1:-1])
        p_implicite = np.eye(N) + theta*A
        u1[1:-1] = npl.solve(p_implicite, p_explicite)
        U.append(u1)
        n += 1
        
    return U

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

x = np.linspace(0, 1, N+2)
line, = ax.plot(x, U[0])


def init():  # only required for blitting to give a clean slate.
    line.set_ydata([np.nan] * len(x))
    return line,


def animate(i):
    line.set_ydata(U[i]) # update the data.
    line.set_label(str(i*dt))
    return line,


ani = animation.FuncAnimation(
    fig, animate, init_func=init, interval=1, blit=True)

ani.save('chaleur.gif', writer='imagemagick')
plt.show()

In [70]:
plt.close('all')

## Exercice 3 : Equations de réaction-diffusion

In [76]:
def u_init(x, mu=0, sigma2=1):
    return ((1/np.sqrt(sigma2*2*pi)) * np.exp(-0.5 * ((x-mu)**2/sigma2)))

x = np.linspace(-10, 10, 1000)
plt.figure()
plt.plot(u_init(x, 0, 5))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
def u_exacte(t, x):
    return 0.5 + 0.5*np.tanh((x + 3*t/np.sqrt(2))/(2*np.sqrt(2)))

x = np.linspace(-10, 10, 100)
t = np.linspace(0, 5, 25)
plt.figure()
plt.plot(u_exacte(0, x))
plt.plot(u_exacte(1, x))
plt.plot(u_exacte(2, x))
plt.plot(u_exacte(3, x))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

T, X = np.meshgrid(t, x)

ax.plot_wireframe(T, X, u_exacte(T, X))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
def mat_A(N):
    diags = [-1, 2, -1]
    return sps.diags(diags, offsets=[-1, 0, 1], shape=[N,N], format="csr")


In [60]:
def sch_impli_expli(u0, dt, dx=0.1, T=0.2, p=2):
    N = int(1/dx) - 1
    x = np.linspace(0, 1, N+2)
    U = [u0(x)]
    n = 0
    A = mat_A(N+2).toarray()
    #modifs de A pour prendre en compte les CLN
    A[0, 0], A[-1, -1] = 1, 1

    while n*dt <= T:
        #u1 = np.zeros_like(U[-1])
        b = U[-1] + np.power(U[-1], p) + U[-1]/dt
        a = np.eye(N+2)/dt + A/(dx**2)
        u1 = npl.solve(a, b)
        #u1[0], u1[-1] = 0, 0
        U.append(u1)
        n += 1
        
    return U

In [ ]:
U = sch_impli_expli(u_init, 0.002, 0.1, 2, 3)
plt.figure()
for k in range(len(U)):
    p = plt.plot(U[k])
plt.show()

In [75]:
U = sch_impli_expli(u_init, 0.005, 0.1, 0.5, 3)
plt.figure()
for k in range(len(U)):
    p = plt.plot(U[k])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …